In [32]:
#!/usr/bin/env python

#  Author: Prakhar Agarwal
#  Date: 12 Jan 2017
# *************************************** #

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.porter import *

In [46]:
train = pd.read_csv("labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)
test = pd.read_csv("testData.tsv",header=0,delimiter="\t",quoting=3)

In [47]:
# Data Cleaning

In [48]:
example1=BeautifulSoup(train["review"][0])
# print train["review"][0]
# print example1.get_text()
letters_only=re.sub("[^a-zA-Z]"," ",example1.get_text())

In [49]:
lower_case = letters_only.lower()
words = lower_case.split()

In [50]:
words = [w for w in words if not w in stopwords.words("english")]
words

[u'stuff',
 u'going',
 u'moment',
 u'mj',
 u'started',
 u'listening',
 u'music',
 u'watching',
 u'odd',
 u'documentary',
 u'watched',
 u'wiz',
 u'watched',
 u'moonwalker',
 u'maybe',
 u'want',
 u'get',
 u'certain',
 u'insight',
 u'guy',
 u'thought',
 u'really',
 u'cool',
 u'eighties',
 u'maybe',
 u'make',
 u'mind',
 u'whether',
 u'guilty',
 u'innocent',
 u'moonwalker',
 u'part',
 u'biography',
 u'part',
 u'feature',
 u'film',
 u'remember',
 u'going',
 u'see',
 u'cinema',
 u'originally',
 u'released',
 u'subtle',
 u'messages',
 u'mj',
 u'feeling',
 u'towards',
 u'press',
 u'also',
 u'obvious',
 u'message',
 u'drugs',
 u'bad',
 u'kay',
 u'visually',
 u'impressive',
 u'course',
 u'michael',
 u'jackson',
 u'unless',
 u'remotely',
 u'like',
 u'mj',
 u'anyway',
 u'going',
 u'hate',
 u'find',
 u'boring',
 u'may',
 u'call',
 u'mj',
 u'egotist',
 u'consenting',
 u'making',
 u'movie',
 u'mj',
 u'fans',
 u'would',
 u'say',
 u'made',
 u'fans',
 u'true',
 u'really',
 u'nice',
 u'actual',
 u'feature

In [64]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]  
    # 6. Stemming (Optional)
    stemmer = PorterStemmer()
    singles = [stemmer.stem(meaningful_words) for meaningful_words in meaningful_words]
    meaningful_words=singles
    #
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [52]:
clean_review = review_to_words( train["review"][0] )
print clean_review

stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obviou messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl stay a

In [54]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in xrange( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews )
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [55]:
clean_train_reviews

[u'stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obviou messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl sta

In [56]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [57]:
vocab = vectorizer.get_feature_names()
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

288 abandon
125 abc
562 abil
1259 abl
83 abomin
69 aborigin
92 abort
63 abound
93 abraham
76 abrupt
60 abruptli
118 absenc
83 absent
1845 absolut
154 absorb
403 absurd
66 abund
398 abus
110 abysm
298 academi
704 accent
781 accept
165 access
344 accid
246 accident
118 acclaim
197 accompani
271 accomplish
311 accord
297 account
349 accur
82 accuraci
204 accus
75 ace
578 achiev
102 acid
109 acknowledg
73 acquaint
97 acquir
971 across
8794 act
3694 action
268 activ
6876 actor
1588 actress
5065 actual
793 ad
409 adam
835 adapt
1147 add
261 addict
499 addit
183 address
148 adequ
440 admir
738 admit
134 admittedli
112 adolesc
162 adopt
226 ador
887 adult
275 advanc
172 advantag
773 adventur
228 advertis
262 advic
195 advis
78 aesthet
419 affair
428 affect
66 affleck
139 afford
126 aforement
343 afraid
212 africa
284 african
197 afternoon
183 afterward
1726 age
79 agenc
86 agenda
455 agent
103 aggress
1033 ago
778 agre
119 ah
396 ahead
289 aid
325 aim
842 air
106 airplan
96 airport
195 aka
67 

In [60]:
print "Training the random forest..."
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"])

Training the random forest...


In [62]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)


Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [63]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model_stemmer.csv", index=False, quoting=3 )